# DAA M04 Data architecture

## Exercise 4.4 - File formats

### Purpose:
This notebook reads and writes different data file formats. A common data set is used for each example.

For each file format, a standard approach is shown. Each of these approaches allows considerable parameterisation to allow for any unusual features of the dataset.

You can review the documentation for each Python module by calling 'help(module_name)' to learn how to deal with such features.


## Packages
This section installs packages that will be required for this exercise/case study.

In [1]:
import struct  # This module converts data into bytes.
import datetime # This is a standard Python module for date types.
import csv
import xml.etree.ElementTree as ET
import pandas as pd
import json

## Data
This section demonstrates ways to import and export different file types.

### Create example data

The example data used for this exercise is created within Python as a list of dictionaries. For dates, the datetime module is used to ensure that the dates are not ambiguous. As you work through the different file formats you should observe the challenges of workin with dates compared to other data types.

In [2]:
# Create a dictionary for each data type.

# Two different formats have been used below to create two
# rows in the dictionary.

data_row_1 = dict()
data_row_1['name'] = 'Robert'
data_row_1['date-of-birth'] = datetime.date(1988, 3, 1)
data_row_1['gender'] = 'Male'

data_row_2 = {'name': 'Suzanne', 'date-of-birth': datetime.date(1986, 8, 11),
              'gender': 'Female'}

# The following code creates the list of dictionaries.
example_data = [data_row_1, data_row_2]

print(example_data)


[{'name': 'Robert', 'date-of-birth': datetime.date(1988, 3, 1), 'gender': 'Male'}, {'name': 'Suzanne', 'date-of-birth': datetime.date(1986, 8, 11), 'gender': 'Female'}]


### Text files

In this section of code, the data is output as a readable text file. Once you have run the code below, you should open the file with a text editor (such as TextEdit in MacOS or Notepad in Windows OS) to view the text file that has been produced. You can download the file from Colab's 'Files' menu on the left-hand menu bar. 

In [3]:
# Create a text file called 'example.txt'.
file = open('example.txt', 'w') # 'w' means 'writing to file'.

# Add data to the text file.
for data_row in example_data:
    nice_sentence = '{0:s} is {2:s} and was born on {1:%B %d, %Y}. '.format(*data_row.values())
    nice_sentence += '\n' # This adds a new line to the end of each data row.
    file.write(nice_sentence) 
file.close()


As you can see, you can create readable text files with Python. Reading this text file back into Python can be achieved with the following code.

In [4]:
# Read 'example.txt' back into Python.
file = open('example.txt', 'r') # 'r' means 'reading from file'.

for line in file:
    print(line, end = '') 

file.close()

Robert is Male and was born on March 01, 1988. 
Suzanne is Female and was born on August 11, 1986. 


What is more difficult (and not shown here) is parsing this 'nice sentence' to get the information that you need from it, such as the date of birth. This is a difficulty associated with using a text format.

Shown below is a more concise way of opening and closing files using a file wrapper. The file is closed once the 'with' code indent block finishes. In most cases, this is a more efficient approach to use when reading text files.

In [5]:
# Read 'example.txt' back into Python using a more efficient approach.
with open('example.txt', 'r') as file:
    for line in file:
        print(line, end ='')

Robert is Male and was born on March 01, 1988. 
Suzanne is Female and was born on August 11, 1986. 


### Binary files

The above exercise is now repeated using a binary file type. Run the write block below to save the data in binary format. Note that most text editors will still be able to open a binary file and interpret it.

In [6]:
# Create a binary file called 'example_binary.txt'.
with open('example_binary.txt', 'wb') as file:  # 'wb' means 'writing to binary file'.
    for data_row in example_data:
        nice_sentence = "{0:s} is {2:s} and was born on {1:%B %d, %Y}.".format(*data_row.values())
        nice_sentence += '\n'
        # The following converts the string into a binary format.
        binary_version = nice_sentence.encode()
        file.write(binary_version)

In [7]:
# Read 'example_binary.txt' back into Python.
with open('example_binary.txt', 'rb') as file: # 'rb' means 'reading from binary file'.
    for line in file:
        print(line.decode('utf8'), end ='')
        # '.decode('utf8')' decodes each line of the binary file into a string object.

Robert is Male and was born on March 01, 1988.
Suzanne is Female and was born on August 11, 1986.


Although the above code processed the data as a binary file, it effectively just copied how the operating system handles text files. The following code is a truer example of writing to and reading from a binary file. Do not worry if you cannot follow all the code below. The important point is that writing and reading binary files can be complex. It is therefore often better to use a standard non-binary data file format.

In [8]:
# Create a function that converts a date into binary format.
def convert_date_to_binary(date):
    binary = struct.pack('@H',date.year)
    binary += struct.pack('@B',date.month)
    binary += struct.pack('@B',date.day)    
    return binary

# Create a function that converts a string into binary format .
def convert_str_to_binary(string):
    # Write the length of each string to the binary file before adding
    # the string to the binary file.
    binary = struct.pack('@H', len(string))
    binary += string.encode()
    return binary

# Create a binary file called 'example.bin'.
with open('example.bin', 'wb') as file:
    for data_row in example_data:
        file.write(convert_str_to_binary(data_row['name']))
        file.write(convert_str_to_binary(data_row['gender']))        
        file.write(convert_date_to_binary(data_row['date-of-birth']))

You should now try to open this binary file with a text editor. What can you see within the file? Depending on the editor, you may be able to make out the names and genders but not the date. The binary file is probably also smaller in size than the text file.

Reading the binary file back into Python is not a straightforward exercise as the length of the two strings must be determined first. The following code demonstrates that this can be done. Again, do not worry about the detail within the code, but instead observe the complexity required to achieve the desired outcome.


In [9]:
# Read the binary file 'example.fin' back into Python.
example_data_read = list()
with open('example.bin', 'rb') as file:
    
    for i in range(2):
        # The first two bytes are the length of the feature 'name'.
        length = struct.unpack('@H', file.read(2))
        # The length information allows us to read the name.
        name = file.read(*length).decode()

        # The next two bytes are the length of the feature 'gender'.
        length = struct.unpack('@H', file.read(2)) 
        gender = file.read(*length).decode()

        # The remaining 4 bytes are used for the feature 'date'.
        year = struct.unpack('@H', file.read(2))
        month = struct.unpack('@B', file.read(1))
        day = struct.unpack('@B', file.read(1))

        # Now convert everything back to a dictionary.
        row_entry = {'name': name, 
                     'date-of-birth': datetime.date(*year, *month, *day),
                     'gender': gender}

        # Add the dictionary to the list.
        example_data_read.append(row_entry)

print(example_data_read)
# The following proves that the output from this code it identical to the original dataset.
print(example_data_read == example_data)


[{'name': 'Robert', 'date-of-birth': datetime.date(1988, 3, 1), 'gender': 'Male'}, {'name': 'Suzanne', 'date-of-birth': datetime.date(1986, 8, 11), 'gender': 'Female'}]
True


### Delimited files

Delimited files are much easier to work with than binary files primarily because readers are available in Python that do most of the work for you. In the code below, Python's csv module is used to write the example data to a csv file.

In [10]:
# Create a comma delimited (csv) file called 'example.csv'.
with open('example.csv', 'w', newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=example_data[0].keys())
    writer.writeheader()
    for row in example_data:
        writer.writerow(row)
        

Open the 'example.csv' file with a text editor and review its format.

The code below reads this csv file back in to Python.

In [11]:
# Read csv file 'example.csv' back into Python.
with open('example.csv', 'r', newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    example_data_read = list(reader)

# Compare the newly created data to the original data.
print(example_data_read)
print(example_data_read==example_data)

[OrderedDict([('name', 'Robert'), ('date-of-birth', '1988-03-01'), ('gender', 'Male')]), OrderedDict([('name', 'Suzanne'), ('date-of-birth', '1986-08-11'), ('gender', 'Female')])]
False


For most purposes, the above code for reading a csv file may be sufficient. However, you will notice that the output from the code above is not exactly the same as the original list of dictionaries in the example data. This is because in some versions of Python, the dictionary is an ordered one not a regular one. This is what 'OrderedDict' refers to in the output. Also the date fields have been read in as text, not as dates. The following code fixes these problems.

In [12]:
# Read csv file 'example.csv' back into Python as a non-ordered dictionary
# and with correct date formats.

# Read csv file 'example.csv' back into Python.
with open('example.csv', 'r', newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    example_data_read = list(reader)

# Convert 'example_data_read' from an ordered dictionary to a regular (non ordered) dictionary.
example_data_read = [dict(row) for row in example_data_read]

# Convert dates from text to date format.
for row in example_data_read:
    row['date-of-birth'] = datetime.datetime.strptime(row['date-of-birth'],'%Y-%m-%d').date()

# Compare the newly created data to the original data.
print(example_data_read)
print(example_data_read==example_data)



[{'name': 'Robert', 'date-of-birth': datetime.date(1988, 3, 1), 'gender': 'Male'}, {'name': 'Suzanne', 'date-of-birth': datetime.date(1986, 8, 11), 'gender': 'Female'}]
True


### XML Files

To create an XML file, you can:
- print an XML file manually; this can be difficult to do for a file of any reasonable length; or
- convert the example data structure into an XML structure and save this XML structure; this is the preferred approach. 

Each of these approaches is shown below.

In [13]:
# Create an XML file 'example.xml' by printing it manually.
row_format = "    <{0:s}>{1:s}</{0:s}>"
with open('example.xml', 'w') as file:
    print('<child_names>', file=file)
    # This line prints the tag '<child_names>' to the XML file.
    for data_row in example_data:
        print('  <child_name>', file=file)
        # This line prints the tag '  <child_name>' to the XML file.
        for key, value in data_row.items():
            # This checks for dates and converts these to a date format.
            if isinstance(value, (datetime.datetime, datetime.date)):
                value = value.isoformat()
            print(row_format.format(key, value), file=file)
        print('  </child_name>', file=file)
    print('</child_names>', file=file)


Once the XML file has been created, the following code loads an XML object. ElementTree is the approach used below. Alternative ways to load an XML object are available.

In [14]:
# Load 'example.xml' back into Python.
tree = ET.parse('example.xml') # Reads in the data as a tree.
root = tree.getroot() # Gets the first part of the tree.

print(root.tag)
for element in root:
    print('  ', element.tag)
    for subelement in element:
        print(' '*4, subelement.tag, ':', subelement.text)


child_names
   child_name
     name : Robert
     date-of-birth : 1988-03-01
     gender : Male
   child_name
     name : Suzanne
     date-of-birth : 1986-08-11
     gender : Female


If you want to convert the data back into a Python data format then the following code will do this. If memory is a concern, you can also delete the XML data using the 'del()' function after running the code below .

In [15]:
# Convert the data tree back into a list of dictionaries.
example_data_read = list()
for element in root:
    data_row = dict()
    for subelement in element:
        data_row[subelement.tag] = subelement.text
    example_data_read.append(data_row)
    
# Convert dates.
for row in example_data_read:
    row['date-of-birth'] = datetime.datetime.strptime(row['date-of-birth'],'%Y-%m-%d').date()

# Compare the newly created list of dictionaries to the original data.
print(example_data_read)
print(example_data_read == example_data)

[{'name': 'Robert', 'date-of-birth': datetime.date(1988, 3, 1), 'gender': 'Male'}, {'name': 'Suzanne', 'date-of-birth': datetime.date(1986, 8, 11), 'gender': 'Female'}]
True


Below is the preferred approach to creating an XML file, in which the example data structure is converted into an XML structure and then saved.

In [16]:
# Create an XML file by converting the example data into an XML structure.

# Create the root of the tree.
example_data_xml = ET.Element('child_names')
# Now create each subelement in turn.
for data_row in example_data:
    element = ET.SubElement(example_data_xml, 'child_name')
    for key, value in data_row.items():
        # This checks for dates and converts them into a date format.
        if isinstance(value, (datetime.datetime, datetime.date)):
            value = value.isoformat()
        # Add a subelement using the key as its tag.
        subelement = ET.SubElement(element, key)
        # Add the value as text belonging to the sub element.
        subelement.text = value

tree = ET.ElementTree(example_data_xml)

# Once created, save the XML structure with a single command.
tree.write('example2.xml')


In [17]:
# Read 'example2.xml' back into Python  using the same code as before.
tree = ET.parse('example2.xml') # Reads in the data as a tree.
root = tree.getroot() # Gets the first part of the tree.

# Convert the XML structure into a list of (non-ordered) dictionaries.
example_data_read = list()
for element in root:
    data_row = dict()
    for subelement in element:
        data_row[subelement.tag] = subelement.text
    example_data_read.append(data_row)
    
# Convert dates.
for row in example_data_read:
    row['date-of-birth'] = datetime.datetime.strptime(row['date-of-birth'],'%Y-%m-%d').date()

# Compare the newly created data to the original data.
print(example_data_read)
print(example_data_read == example_data)

[{'name': 'Robert', 'date-of-birth': datetime.date(1988, 3, 1), 'gender': 'Male'}, {'name': 'Suzanne', 'date-of-birth': datetime.date(1986, 8, 11), 'gender': 'Female'}]
True


### Excel files

Excel files are stored in XML format using an open format. An open format is a file format for storing digital data, defined by a published specification usually maintained by a standards organisation and which can be used and implemented by anyone.

The code below shows how to write and read such a file using the Pandas module.

Pandas also has methods for csv and JSON files as well as some less known file types which you may want to use for your projects.

In [18]:
# Write a file 'example.xlsx' to Excel.
pd.DataFrame(example_data).to_excel('example.xlsx', sheet_name='example', index=False)


The next cell block reads the Excel file back into Python. Note that in the code below, Pandas is able to automatically identify and convert the dates. 

In [19]:
# Load the Excel file 'example.xlsx' back into Python.
example_data_read = pd.read_excel('example.xlsx', sheet_name='example')

print(example_data_read)
print('\n')
print('Pandas has read in the date field as type:')
print(example_data_read['date-of-birth'].dtype)
print('\n')

# Convert the data into a Pandas Dataframe.
example_data_pd = pd.DataFrame(example_data)
print(example_data_pd)


      name date-of-birth  gender
0   Robert    1988-03-01    Male
1  Suzanne    1986-08-11  Female


Pandas has read in the date field as type:
datetime64[ns]


      name date-of-birth  gender
0   Robert    1988-03-01    Male
1  Suzanne    1986-08-11  Female


### JSON files

JSON files are relatively easy to work with in Python as a JSON writer and reader module is provided. The date format does create a problem as there is no automatic serialiser for this. This can be overcome by creating one for the purpose and using that instead of the default.  

You can open the JSON file output in a web browser.

In [20]:
# Create a JSON file 'example.json'.

# Define a function to create a JSON file.
def json_dates(obj):
    # Convert date fields to date format.
    if isinstance(obj, (datetime.datetime, datetime.date)):
        return obj.isoformat()
 
    # The following line deals with errors from the default routine.
    raise TypeError ("Type %s not serializable" % type(obj))

with open('example.json', 'w') as file:
    json.dump(example_data, file, default=json_dates, indent = 2)


The next section of code reads this JSON file back into Python. Note the additional coding that is required to handle the date conversion.

In [ ]:
# Read 'example.json' back into Python.
with open('example.json', 'r') as file:
    example_data_read = json.load(file)
    
# Format dates correctly
for row in example_data_read:
    row['date-of-birth'] = datetime.datetime.strptime(row['date-of-birth'],'%Y-%m-%d').date()
    
print(example_data_read)
print(example_data_read == example_data)

[{'name': 'Robert', 'date-of-birth': datetime.date(1988, 3, 1), 'gender': 'Male'}, {'name': 'Suzanne', 'date-of-birth': datetime.date(1986, 8, 11), 'gender': 'Female'}]
True
